In [1]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt

d:\producto\anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


Curvas u

In [2]:
# tiempo de muestreo [segundos]
Ts = 0.01

In [3]:
N = 2000
x     = np.zeros((N,1), dtype=float)
x_dot = np.zeros((N,1), dtype=float)
x_req = np.zeros((N,1), dtype=float)
u = np.zeros((N,1), dtype=float)

In [25]:
U = 0.1;
x0_list = [1, 10, 20]

plt.plot(x,x_dot,'.-')


for x0 in x0_list:
    
    for k in range(0,x.size-1):
        x[k+1]     = x[k] + Ts*x_dot[k]
        x_dot[k+1] = x_dot[k] + Ts * U

    plt.plot(x,x_dot,'--')

# posicion    
plt.xlabel('sample')
plt.ylabel('x [m]')
plt.grid()
# plt.xlim([0, 1])
plt.draw()
plt.savefig('test_1.png')


SyntaxError: invalid syntax (<ipython-input-25-6459122f0b0d>, line 7)

In [26]:
x0

[1, 2, 3]

In [27]:
for k in x0:
    print(k)

1
2
3
